In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

# register at https://scihub.copernicus.eu/dhus/#/home
user = ''
password = ''
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

In [2]:
footprint = geojson_to_wkt(read_geojson('../glass-fire/map.geojson'))


products = api.query(
    footprint, date=('20200920', "20201001"), platformname='Sentinel-2', producttype="S2MSI1C"
)

product_df = api.to_dataframe(products)
product_df.head()

,title,link,link_alternative,link_icon,summary,ondemand,datatakesensingstart,beginposition,endposition,ingestiondate,...,s2datatakeid,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,identifier,uuid,granuleidentifier,datastripidentifier
f48fb0ee-a209-4b66-b5ec-5c3a66190412,S2B_MSIL1C_20200929T190109_N0209_R013_T10SDJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-09-29T19:01:09.024Z, Instrument: MS...",false,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 22:30:30.319,...,GS2B_20200929T190109_018629_N02.09,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20200929T190109_N0209_R013_T10SDJ_2...,f48fb0ee-a209-4b66-b5ec-5c3a66190412,S2B_OPER_MSI_L1C_TL_VGS1_20200929T213343_A0186...,S2B_OPER_MSI_L1C_DS_VGS1_20200929T213343_S2020...
71d5a022-f6ad-4cbc-ab41-ba782a8481ee,S2B_MSIL1C_20200929T190109_N0209_R013_T10SEJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-09-29T19:01:09.024Z, Instrument: MS...",false,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 22:30:17.643,...,GS2B_20200929T190109_018629_N02.09,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20200929T190109_N0209_R013_T10SEJ_2...,71d5a022-f6ad-4cbc-ab41-ba782a8481ee,S2B_OPER_MSI_L1C_TL_VGS1_20200929T213343_A0186...,S2B_OPER_MSI_L1C_DS_VGS1_20200929T213343_S2020...
bd491342-5a1b-4e93-846b-74d4052bdfbe,S2B_MSIL1C_20200929T190109_N0209_R013_T10SDH_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-09-29T19:01:09.024Z, Instrument: MS...",false,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 22:28:26.919,...,GS2B_20200929T190109_018629_N02.09,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20200929T190109_N0209_R013_T10SDH_2...,bd491342-5a1b-4e93-846b-74d4052bdfbe,S2B_OPER_MSI_L1C_TL_VGS1_20200929T213343_A0186...,S2B_OPER_MSI_L1C_DS_VGS1_20200929T213343_S2020...
f060901c-8bf6-47c4-be2f-fd7a253e7a91,S2B_MSIL1C_20200929T190109_N0209_R013_T10SEH_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-09-29T19:01:09.024Z, Instrument: MS...",false,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 22:28:17.463,...,GS2B_20200929T190109_018629_N02.09,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20200929T190109_N0209_R013_T10SEH_2...,f060901c-8bf6-47c4-be2f-fd7a253e7a91,S2B_OPER_MSI_L1C_TL_VGS1_20200929T213343_A0186...,S2B_OPER_MSI_L1C_DS_VGS1_20200929T213343_S2020...
33cc42a2-3f3b-4a1c-bcdd-b4ea19d753ee,S2B_MSIL1C_20200926T185049_N0209_R113_T10SEJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-09-26T18:50:49.024Z, Instrument: MS...",false,2020-09-26 18:50:49.024,2020-09-26 18:50:49.024,2020-09-26 18:50:49.024,2020-09-26 23:20:57.362,...,GS2B_20200926T185049_018586_N02.09,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20200926T185049_N0209_R113_T10SEJ_2...,33cc42a2-3f3b-4a1c-bcdd-b4ea19d753ee,S2B_OPER_MSI_L1C_TL_VGS2_20200926T210259_A0185...,S2B_OPER_MSI_L1C_DS_VGS2_20200926T210259_S2020...


In [3]:
## A product is only available within a year on the server. After that it needs to be requested. Only one request per day...
# for product_id in products:
#     product_info = api.get_product_odata(product_id)
#     print(product_id, product_info["Online"])

In [ ]:
api.download_all(products, directory_path='glass-fire/data/')